In [154]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor

In [155]:
df=pd.read_csv('adult.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
5,34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,<=50K
6,38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40,United-States,<=50K
7,74,State-gov,88638,Doctorate,16,Never-married,Prof-specialty,Other-relative,White,Female,0,3683,20,United-States,>50K
8,68,Federal-gov,422013,HS-grad,9,Divorced,Prof-specialty,Not-in-family,White,Female,0,3683,40,United-States,<=50K
9,41,Private,70037,Some-college,10,Never-married,Craft-repair,Unmarried,White,Male,0,3004,60,?,>50K


In [156]:
df.replace('?',np.nan,inplace=True)
df.dropna(subset=['income'],inplace=True)
x=df.drop('income',axis=1)
y=df['income']

In [157]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,train_size=0.8,random_state=42)


In [158]:
num_col=x.select_dtypes(exclude=['object']).columns
cat_col=x.select_dtypes(include=['object']).columns

In [159]:
prepro=ColumnTransformer(
    transformers=[
        ('pre',Pipeline(steps=[
            ('simp',SimpleImputer()),
            ('std',StandardScaler())]),num_col),
        ('pre2',Pipeline(steps=[
            ('onehot',OneHotEncoder()),
            ('simp2',SimpleImputer(strategy='most_frequent'))
        ]),cat_col)
],remainder='passthrough'
)

In [160]:
pipe=Pipeline(
    steps=[
        ('prepro',prepro),
        ('model',DecisionTreeClassifier())
    ]
)


In [161]:
pipe2=Pipeline(
    steps=[
        ('prepro',prepro),
        ('model2',RandomForestClassifier(n_estimators=10,random_state=42))
    ]
)

In [162]:
pipe.fit(xtrain,ytrain)

,steps,"[('prepro', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('pre', ...), ('pre2', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [163]:
pipe.score(xtest,ytest)

0.8140641793336404

In [164]:

    
params={
    'model2__criterion':['entropy','gini'],
    'model2__min_samples_split':[2,3],
    'model2__max_depth':[5,10,],
    'model2__min_samples_leaf':[2,3,]
}


In [165]:
grid=GridSearchCV(pipe2,params,cv=5,n_jobs=-1)

In [166]:
grid.fit(xtrain,ytrain)

c:\Users\Varun\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(


,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'model2__criterion': ['entropy', 'gini'], 'model2__max_depth': [5, 10], 'model2__min_samples_leaf': [2, 3], 'model2__min_samples_split': [2, 3]}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('pre', ...), ('pre2', ...)]"


In [167]:
grid.score(xtest,ytest)

0.8278826961461692